Redes Neuronales:Es un tipo de algoritmo de aprendizaje automatico, ispirado en el cerebro, estan diseñados para aprender de datos, se pueden utilizar para aprender patrones y demas, su arquitectura es muy flexible y puede adaptarse a muchos problemas y dominios, algunas desventajas de este pueden darse cuando se trabaja con datasets con bajas cantidades de datos ya que se necesitan muchos datos para un aprendiazaje mas profundo y optimo, otra desventaja puede ser el tiempo de entrenamiento que requiere ya, que al utilizarse muchos datos tardara conciderablemente para aprender, para su ejemplo se utilizara el dataset: https://www.kaggle.com/datasets/calvom/agricultura
que basicamente se trata de que calidad tiene la agricultura de ciertas regiones en base a ciertas caracteristicas se clasifican en alta media y baja

El cuadernillo se copio de Sencio

In [138]:
#PRIMERO IMPORTAMOS LAS LIBRERIAS NECASARIAS PARA PODER TRANAJAR EN LA CLASEFICACION
# utilizado para manejos de directorios y rutas
import os
#impotamos la libreria de pandas para el manejo de los datos:
import pandas as pd
# Computacion vectorial y cientifica para python
import numpy as np

# Librerias para graficación (trazado de gráficos)
from matplotlib import pyplot
from mpl_toolkits.mplot3d import Axes3D  # Necesario para graficar superficies 3D
from sklearn.preprocessing import LabelEncoder

# llama a matplotlib a embeber graficas dentro de los cuadernillos
%matplotlib inline

Definimos la clase MLP (red neuronal)como secuencia de capas, las cuales se pasaran a un lista  a la clase MLP(Multilayer Perceptron)

Se realiza la propagacion hacie adelante a travez de todas las capas del modelo.

In [139]:
class MLP:
    def __init__(self, layers): #le pasamo una lista de capas
        # el MLP es una lista de capas
        self.layers = layers

    def __call__(self, x): #llamamos al perceptron multicapa
        for layer in self.layers: #iteraremos por esta lista de capas y a cada capa
            x = layer(x)         #le pasaremos las x o inputs y sus salidas seran las entradas
        return x                #de la suiguente capa, y asi este bucle correra por todas las capas que se tenga

Implentacion de las capas
Aqui se proporciona una estructura basica para las capas, y se puden implementar metodos para operaciones como calculos de activacion, gradientes y actualizacion de parametros

In [140]:
class Layer(): #la clase base que contendra lo elementos y la funcionalidad de la capa.
    def __init__(self):
        self.params = [] #contendra los elementos entrenables de la capa
        self.grads = [] #contendra los gradientes que se aplicaran a los parametos

    def __call__(self, x): #devuelve las entradas como si fueran salidas
        return x

    def backward(self, grad):
        # cada capa, calculará sus gradientes
        # y los devolverá para las capas siguientes
        return grad

    def update(self, params):
        # si hay parámetros, los actualizaremos
        # con lo que nos de el optimizador
        return

Ahora se definen las capas que se utilizaran para este modelo
Es un capa lineal , esto es un perceptron y un perceptorn multicapa es un secuencia de perceptrones
(2 / (d_in + d_out)) se utiliza para evitar problemas de saturacion y desvanecimiento del gradiente bias(reisgo)
el aprendizaje de cada capa se almacena en self.w, self.b donde se guardan los pesos y el bias(inclinacion de los pesos)

In [141]:
class Linear(Layer):
    def __init__(self, d_in, d_out): # d_in=dimecion de entradas de la capas
        # pesos de la capa           #d_out=dimecion de salidas de la capa
        self.w = np.random.normal(loc=0.0, #basicamente es una matriz de pesos a la cual
                                  scale=np.sqrt(2/(d_in+d_out)),#multiplicaremos por los inputs le sumaremos el
                                  size=(d_in, d_out))#bayas y el resultado es lo que devuelve la capa
        self.b = np.zeros(d_out)

    def __call__(self, x):  #recibimos entradas, guardamos
        self.x = x
        self.params = [self.w, self.b]
        # salida del preceptrón
        return np.dot(x, self.w) + self.b #devolvemos el producto de la matriz por las entradas + el bias

    def backward(self, grad_output):
        #calcular gradientes para pasar a la capa siguiente (BACKPROP)
        grad = np.dot(grad_output, self.w.T) #es multiplicar la matriz de pesos por el gradiente que nos llega de la capa anterior
        self.grad_w = np.dot(self.x.T, grad_output) #gradientes respecto a los parametros(pesos) producto de las entradas por el gradinete de la suiguiente capa
        # gradientes para actualizar pesos
        self.grad_b = grad_output.mean(axis=0)*self.x.shape[0] #finalmente se calcula el gradiente respecto al bias
        self.grads = [self.grad_w, self.grad_b]
        return grad

    def update(self, params): #actulliza los parametros de una capa por los paramtros calculados
        self.w, self.b = params #asigna los nuevos valores de pesos y el bias en los atributos de la capa

Funciones de activacion:
RELU: BAsicamente la funcion relu lo que hace es que: si x es mayor a 0 la funcion devuelve el mismo valor y si es menor o igual a cero devuelve 0
Sigmoide:nos permitira devolver que siempre estar entre 1 y 0

In [142]:
class ReLU(Layer):#recibe entradas
    def __call__(self, x): #recibe entradas
        self.x = x #guaradar las entradas para el bacward
        return np.maximum(0, x)# nos devolvera el valor maximo entre los valores los inputs y 0

    def backward(self, grad_output): #nos llega el gradiente de la capa suiguiente
        grad = self.x > 0 #lo multiplicamos por la derivada de la relu
        return grad_output*grad #y devolvemos el resultado para que sea utilizado por la capa anterior para calcular su gradientes

def sigmoid(x):
  return 1 / (1 + np.exp(-x)) #sigmoide F(x)uno sobre uno mas exponecial negativa de x

def softmax(x):#devuleve una distribucion de probabilidad
    return np.exp(x) / np.exp(x).sum(axis=-1,keepdims=True) #softmax(x_i) = exp(x_i) / sum(exp(x_j))

class Sigmoid(Layer):
    def __call__(self, x):
        self.x = x
        return sigmoid(x)

    def backward(self, grad_output):
        grad = sigmoid(self.x)*(1 - sigmoid(self.x)) #se hace su derivad respecto a la sigmoide
        return grad_output*grad

Optimizador: descenso por el gradiente
se iterara por todas capas del MLP actualizado los parametros lr(Taza de aprendizaje)

In [143]:
class SGD():
    def __init__(self, net, lr):
        self.net = net
        self.lr = lr

    def update(self): #actualiza todos los parametros de la capa
        for layer in self.net.layers:#itera recursivamente por todas las capas de la red neuronal
            layer.update([ #se llama a su propia funcion uptade le pasamos los paramtros actualizados y los aplica
                params - self.lr*grads #aplicamos la regla de actualizacion
                for params, grads in zip(layer.params, layer.grads) #aqui iteramos por los paramtrosy gradientes de cada capa
            ])

Funciones de Pérdida:Primero se implement la clase lossy luego CrossEntropy hereda de ella

In [144]:
class Loss():
    def __init__(self, net): # en el inicializador le pasamos el  MLP
        self.net = net # se lo guardara (referencia a la red nueronal)

    def backward(self): #esta funcion se encarga de desencadenar todo el calculo de las derivadas
        # calcula la derivada de la  function de perdida con respecto
        # a la salida del MLP
        grad = self.grad_loss() #apartir de ese gradiente se desencade el proceso de propagacion
        # BACKPROPAGATION
        for layer in reversed(self.net.layers): #iteramos por todas las capas pero desde la ultima hasta la primera
            grad = layer.backward(grad) #utilizando el gradiente de cada capa como entrada del greadiente del anterior
                                        #se calcula su  gradeinte se lo pasa al anterior y asi hasta llegar a las entradas

class MSE(Loss):
    def __call__(self, output, target):
        self.output, self.target = output, target.reshape(output.shape)
        loss = np.mean((self.output - self.target)**2)
        return loss.mean()

    def grad_loss(self):
        return self.output -  self.target

class BCE(Loss):
    def __call__(self, output, target):
        self.output, self.target = output, target.reshape(output.shape)
        loss = - np.mean(self.target*np.log(self.output) - (1 - self.target)*np.log(1 - self.output))
        return loss.mean()

    def grad_loss(self):
        return self.output -  self.target
#clasficacion multioclase
class CrossEntropy(Loss): #se hereeda de la clase loss calcula la funcion de perdida en base a sus gradientes
    def __call__(self, output, target):
        self.output, self.target = output, target
        logits = output[np.arange(len(output)), target]
        loss = - logits + np.log(np.sum(np.exp(output), axis=-1))
        loss = loss.mean()
        return loss
       #basicamente lo que hace la calse CrossEntropy es que hace el calculo de la funcion de perdida
       #y el calculo de su gradiente, lo que nos permite obtner la funcion de perdida en el entrenamiento
       #para poder realizar la propagacion hacia atras y poder actualizar los pesos de la red
    def grad_loss(self):
        answers = np.zeros_like(self.output)
        answers[np.arange(len(self.output)), self.target] = 1
        return (- answers + softmax(self.output)) / self.output.shape[0]

In [145]:
datos = pd.read_csv('agricultura.csv', delimiter=',')

<ipython-input-145-ff3ad8f8eef0>:1: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  datos = pd.read_csv('agricultura.csv', delimiter=',')


In [146]:
datos

,ID Provincia,Provincia,ID Departamento,Departamento,Id Cultivo,Cultivo,ID Campa�a,Campana,Temperatura (Ce),Sup. Sembrada (Ha),Sup. Cosechada (Ha),Producci�n (Tn),Rendimiento (Kg/Ha),Calidad
0,6,BUENOS AIRES,854,25 DE MAYO,1,Ajo,1,1969/70,23,3.00,3.00,10,3.333,Alta
1,6,BUENOS AIRES,854,25 DE MAYO,1,Ajo,2,1970/71,21,1.00,1.00,3,3.000,Promedio
2,6,BUENOS AIRES,14,ADOLFO GONZALES CHAVES,1,Ajo,1,1969/70,30,15.00,15.00,82,5.467,Promedio
3,6,BUENOS AIRES,14,ADOLFO GONZALES CHAVES,1,Ajo,2,1970/71,31,10.00,10.00,55,5.500,Alta
4,6,BUENOS AIRES,14,ADOLFO GONZALES CHAVES,1,Ajo,3,1971/72,26,8.00,8.00,44,5.500,Alta
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132764,54,MISIONES,112,SAN PEDRO,31,Yerba mate,40,2008/09,31,5.50,4.80,23.76,4.950,Promedio
132765,54,MISIONES,112,SAN PEDRO,31,Yerba mate,41,2009/10,31,5.50,4.80,24.48,5.100,Promedio
132766,54,MISIONES,112,SAN PEDRO,31,Yerba mate,43,2011/12,31,5.85,5.15,23.175,4.500,Baja
132767,54,MISIONES,112,SAN PEDRO,31,Yerba mate,45,2013/14,31,5.85,4.85,26.19,5.400,Baja


In [147]:
columnas_eiliminar = ['Producci�n (Tn)']
data = datos.drop(columnas_eiliminar, axis=1)

In [148]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132769 entries, 0 to 132768
Data columns (total 13 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   ID Provincia         132769 non-null  int64  
 1   Provincia            132769 non-null  object 
 2   ID Departamento      132769 non-null  int64  
 3   Departamento         132769 non-null  object 
 4   Id Cultivo           132769 non-null  int64  
 5   Cultivo              132769 non-null  object 
 6   ID Campa�a           132769 non-null  int64  
 7   Campana              132769 non-null  object 
 8   Temperatura (Ce)     132769 non-null  int64  
 9   Sup. Sembrada (Ha)   132769 non-null  float64
 10  Sup. Cosechada (Ha)  132769 non-null  float64
 11  Rendimiento (Kg/Ha)  132769 non-null  float64
 12  Calidad              132769 non-null  object 
dtypes: float64(3), int64(5), object(5)
memory usage: 13.2+ MB


In [149]:
columnas_categoricas = data.select_dtypes(include=['object']).columns

In [150]:
#Procesamiento de frases
for columna in columnas_categoricas:
  le = LabelEncoder()
  data[columna] = le.fit_transform(data[columna])

In [151]:
data

,ID Provincia,Provincia,ID Departamento,Departamento,Id Cultivo,Cultivo,ID Campa�a,Campana,Temperatura (Ce),Sup. Sembrada (Ha),Sup. Cosechada (Ha),Rendimiento (Kg/Ha),Calidad
0,6,0,854,4,1,0,1,0,23,3.00,3.00,3.333,0
1,6,0,854,4,1,0,2,1,21,1.00,1.00,3.000,2
2,6,0,14,7,1,0,1,0,30,15.00,15.00,5.467,2
3,6,0,14,7,1,0,2,1,31,10.00,10.00,5.500,0
4,6,0,14,7,1,0,3,2,26,8.00,8.00,5.500,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
132764,54,12,112,349,31,31,40,39,31,5.50,4.80,4.950,2
132765,54,12,112,349,31,31,41,40,31,5.50,4.80,5.100,2
132766,54,12,112,349,31,31,43,42,31,5.85,5.15,4.500,1
132767,54,12,112,349,31,31,45,44,31,5.85,4.85,5.400,1


In [152]:
X = data.iloc[:, :12]
Y = data.iloc[:, 12]
X_mean, X_std = X.mean(axis=0), X.std(axis=0) #X.mean Se calcula la media de cada columna a lo largo de las filas, y X.std la desciacion estandar de cada columna
X_norm = (X - X_mean) / X_std #se calcula la media restando la media de cada columna y dividiendo por su desviacion

X.shape, Y.shape
print(X)
print("-----------------")
print(Y)

        ID Provincia  Provincia  ID Departamento  Departamento  Id Cultivo  \
0                  6          0              854             4           1   
1                  6          0              854             4           1   
2                  6          0               14             7           1   
3                  6          0               14             7           1   
4                  6          0               14             7           1   
...              ...        ...              ...           ...         ...   
132764            54         12              112           349          31   
132765            54         12              112           349          31   
132766            54         12              112           349          31   
132767            54         12              112           349          31   
132768            54         12              112           349          31   

        Cultivo  ID Campa�a  Campana  Temperatura (Ce)  Sup. Se

In [153]:
etiquetas_unicas = np.unique(Y)
cantidad_etiquetas = len(etiquetas_unicas)

print("Etiquetas únicas:", etiquetas_unicas)
print("Cantidad de etiquetas:", cantidad_etiquetas)

Etiquetas únicas: [0 1 2]
Cantidad de etiquetas: 3


In [154]:
D_in, H, D_out = 12, 100, 3  #se tiene 12 x entradas 100nueronas para cada capa oculta 3 salidas que serian baja media alta
mlp = MLP([
    Linear(D_in, H), # capa de entrada
    ReLU(),        # Funcionde activacion
    Linear(H, H),#capa oculta
    Sigmoid(),
    #Linear(H, H),
    #ReLU(),
    Linear(H, D_out) #capa de salida
])

optimizer = SGD(mlp, lr=0.02) #como optimizador utilizamos el deseso por el gradiente
loss = CrossEntropy(mlp) #definimos como funcion de perdida  al CrossEntropy

epochs = 100 #se entrena a lo largo de iteraciones completas del conjunto de datos del entrenamiento
batch_size = 10 #Significa que se utilizo 10 lotes de muestra para cada actualizacion de los pesos del entrenamiento

batches = len(X) // batch_size #132769
log_each = 10
l = []
for e in range(1,epochs+1): #iteramos por todos los epocs
    _l = []
    for b in range(batches): #Itermanos por todos los bache
        x = X_norm[b*batch_size:(b+1)*batch_size] #tomamos los datos que corresponden a cada X
        y = Y[b*batch_size:(b+1)*batch_size] #tomamos los datos que corresponden a cada y
        y_pred = mlp(x) #aqui se calcula la salida del modelo
        _l.append(loss(y_pred, y)) #aqui se guarda la funcion de perdida mientras se entrena
        loss.backward()#se calcula todos los gradientes
        optimizer.update() #acemos la actualizacion de los pesos
    l.append(np.mean(_l))
    if not e % log_each:
        print(f'Epoch {e}/{epochs}, Loss: {np.mean(l):.4f}')

Epoch 10/100, Loss: 1.0602
Epoch 20/100, Loss: 1.0579
Epoch 30/100, Loss: 1.0569
Epoch 40/100, Loss: 1.0563
Epoch 50/100, Loss: 1.0558
Epoch 60/100, Loss: 1.0555
Epoch 70/100, Loss: 1.0552
Epoch 80/100, Loss: 1.0550
Epoch 90/100, Loss: 1.0548
Epoch 100/100, Loss: 1.0546


In [155]:
correct_predictions = 0
total_predictions = 0
X_test=X = data.iloc[132668:, :12]
Y_test=Y = data.iloc[132668:, 12]
X_mean, X_std = X_test.mean(axis=0), X.std(axis=0)
X_norm_test = (X - X_mean) / X_std
for i in range(len(X_test)):
    x_test = X_norm_test.iloc[i]
    y_test = Y_test.iloc[i]
    y_pred = mlp(x_test)

    predicted_class = np.argmax(y_pred)  # Obtenemos el índice de la clase con mayor probabilidad
    true_class = y_test

    if predicted_class == true_class:
        correct_predictions += 1

    total_predictions += 1

accuracy = correct_predictions / total_predictions
print(f'Accuracy on Test Set: {accuracy:.2f}')

Accuracy on Test Set: 0.31
